<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/02_Mnist_tanh2lys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

class MnistClassificationTwoHiddenLayerModel(tf.keras.Model):
    def __init__(self, numOf1HiddenNode, numOf2HiddenNode, numOfOutput):

        super(MnistClassificationTwoHiddenLayerModel, self).__init__()

        self.hiddenLayer1 = tf.keras.layers.Dense(numOf1HiddenNode,
                                                  activation='tanh',
                                                  kernel_initializer=self.weight_initialization(),
                                                  bias_initializer=self.weight_initialization()
                                                  )
        self.dropout1 = tf.keras.layers.Dropout(0.5)

        self.hiddenLayer2 = tf.keras.layers.Dense(numOf2HiddenNode,
                                                  activation='tanh',
                                                  kernel_initializer=self.weight_initialization(),
                                                  bias_initializer=self.weight_initialization())
        self.dropout2 = tf.keras.layers.Dropout(0.5)

        self.outPut = tf.keras.layers.Dense(numOfOutput,
                                            activation=None,
                                            kernel_initializer=self.weight_initialization(),
                                            bias_initializer=self.weight_initialization()
                                            )
    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1., seed=None)

    def call(self, x):
        y1 = self.hiddenLayer1(x)
        y2 = self.hiddenLayer2(y1)
        y3 = self.outPut(y2)
        return tf.nn.softmax(y3)

@tf.function
def cross_entropy(ypred, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=ypred, labels=y))
    # return tf.reduce_mean(-tf.reduce_sum(y*tf.math.log(ypred), axis=[1]))

@tf.function
def backward(model, optimizer, x, y):
    with tf.GradientTape() as gt:
        yPred = model(x)
        loss = cross_entropy(yPred, y)
    gradz = gt.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradz, model.trainable_variables))

@tf.function
def accuracy(yPred, y):
    correction = tf.equal(tf.argmax(yPred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(correction, tf.float32))
    return acc

def train_learning(epochs, numBatch, numOfTrain, trainData, model, optimizer, lossFrequence, XTest, yTest):
    loss_history = []
    acc_history = []

    for epoch in range(epochs):
        avgLoss = 0.
        totBatch = int(numOfTrain/numBatch)
        for bx, by in trainData:
            backward(model, optimizer, bx, by)
            yPreds = model(bx)
            lossBatch = cross_entropy(yPreds, by)
            avgLoss += lossBatch/totBatch

        loss_history.append(avgLoss)

        if epoch % lossFrequence == 0:
            print(f"Epoch: {epoch+1} ===> Loss: {avgLoss}")

        yPred = model(XTest)
        acc = accuracy(yPred, yTest)
        acc_history.append(acc)

        print(f"Epoch: {epoch+1} ===> Accuracy: {acc}")

    plt.figure(figsize=(6, 4))

    plt.plot(loss_history, label="Loss")
    plt.plot(acc_history, label="Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.title("Loss and Accuracy Progress")
    plt.legend()

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    (XTrain, yTrain), (XTest, yTest) = tf.keras.datasets.mnist.load_data()
    XTrain, XTest = XTrain.astype('float32'), XTest.astype('float32')
    XTrain, XTest = XTrain.reshape([-1, 784]), XTest.reshape([-1, 784])
    XTrain, XTest = XTrain/255., XTest/255.
    yTrain, yTest = tf.one_hot(yTrain, depth=10), tf.one_hot(yTest, depth=10)

    learningRate = 0.001
    numOfEpochs = 15
    numOfBatch = 128
    lossFrequence = 1
    numOfInput = 784
    numOf1HiddenNode = 512
    numOf2HiddenNode = 256
    numOfOutput = 10
    numOfTrain = XTrain.shape[0]

    trainData = tf.data.Dataset.from_tensor_slices((XTrain, yTrain))
    trainData = trainData.shuffle(60000).batch(numOfBatch)

    model = MnistClassificationTwoHiddenLayerModel(numOf1HiddenNode, numOf2HiddenNode, numOfOutput)
    optimizer = tf.optimizers.Adam(learningRate)

    train_learning(numOfEpochs, numOfBatch, numOfTrain, trainData, model, optimizer, lossFrequence, XTest, yTest)